# Install libs

In [11]:
!python -m pip install -U pip httpx beautifulsoup4

# Problem statement:
Problem
You are given a published Google Doc like this one that contains a list of Unicode characters and their positions in a 2D grid. Your task is to write a function that takes in the URL for such a Google Doc as an argument, retrieves and parses the data in the document, and prints the grid of characters. When printed in a fixed-width font, the characters in the grid will form a graphic showing a sequence of uppercase letters, which is the secret message.

The document specifies the Unicode characters in the grid, along with the x- and y-coordinates of each character.

The minimum possible value of these coordinates is 0. There is no maximum possible value, so the grid can be arbitrarily large.

Any positions in the grid that do not have a specified character should be filled with a space character.

You can assume the document will always have the same format as the example document linked above.

For example, the simplified example document linked above draws out the letter 'F':


█▀▀▀
█▀▀
█   


Note that the coordinates (0, 0) will always correspond to the same corner of the grid as in this example, so make sure to understand in which directions the x- and y-coordinates increase.

Specifications
Your code must be written in Python (preferred), JavaScript, TypeScript, Java, Kotlin, C#, C++, Go, Rust, Swift or Ruby.

You may use external libraries.

You may write helper functions, but there should be one function that:

1. Takes in one argument, which is a string containing the URL for the Google Doc with the input data, AND

2. When called, prints the grid of characters specified by the input data, displaying a graphic of correctly oriented uppercase letters.

# Import libs

In [12]:
import httpx
from bs4 import BeautifulSoup

# Get data from url

In [13]:
async def get_html(url: str) -> str:
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        print("Fetched content from url successfully!")
        return response.text

# Collect coordinates

In [14]:
def collect_coordinates(html: str) -> dict[tuple[int, int], str]:
  soup: BeautifulSoup = BeautifulSoup(html, "html.parser")
  coordinates: dict[tuple[int, int], str] = {}
  # Find all table rows but skip the first row since it'll be a title row
  # for 3 columns
  for row in soup.find_all("tr")[1:]:
    x, char, y = row.find_all("td")
    coordinates[(int(x.text), int(y.text))] = char.text

  return coordinates

# Sort coordinates

In [15]:
def sort_coordinates(coordinates: dict[tuple[int, int], str]) -> dict[tuple[int, int], str]:
  # Sort the coordinates
  sorted_coordinates: dict[tuple[int, int], str] = dict(
      sorted(coordinates.items(), key=lambda x: x[0][0])
  )
  print(f"*** Collected coordinates: {sorted_coordinates=}")

  return sorted_coordinates

# Print graphics

In [16]:
def print_graphics_from_coords(
    sorted_coordinates: dict[tuple[int, int], str]
) -> None:
  # Here we make one pass to get the maximum X & Y coordinates from the
  # keys of the coordinates dict
  # It is mentioned that the minimum value of X and Y is 0.
  # Hence we start with a max of each X and Y as 0.
  max_x: int = 0
  max_y: int = 0
  for (x, y) in sorted_coordinates.keys():
    if x > max_x:
        max_x = x
    if y > max_y:
        max_y = y

  # Here we iterate in such a manner that the characters are placed
  # accurately in the grid without getting inverted characters originally
  # We start with maximum Y decreasing till origin
  # and within we iterate X from origin till its maximum
  for y in range(max_y, -1, -1):
    for x in range(max_x + 1):

      # We print the character directly horizontally
      # without a new line for the same row
      if (char := sorted_coordinates.get((x,y), None)):
        print(char, end="")

      else:
        # We print a space as mentioned in the problem statement that
        # a space is needed if coordinates do not have any characters
        # associated. This is for pretty recognition of the printable
        # graphics in the grid.
        # Print without a new line for the same row
        print(" ", end="")

    # After printing all characters in a row, print a newline
    print()

In [17]:
# This is the main function to pass the input URL
# and check the printable graphics
async def print_graphics(url: str) -> str:
  html = await get_html(url=url)
  sorted_coordinates = sort_coordinates(
      coordinates=collect_coordinates(html=html)
  )
  print_graphics_from_coords(sorted_coordinates=sorted_coordinates)

In [19]:
# Ask the user for the url to scan/parse
url: str = input("Enter the url to scan/parse: ").strip()
await print_graphics(url=url)

Enter the url to scan/parse: https://docs.google.com/document/d/e/2PACX-1vRPzbNQcx5UriHSbZ-9vmsTow_R6RRe7eyAU60xIF9Dlz-vaHiHNO2TKgDi7jy4ZpTpNqM7EvEcfr_p/pub
Fetched content from url successfully!
*** Collected coordinates: sorted_coordinates={(0, 4): '█', (0, 6): '█', (0, 1): '█', (0, 2): '█', (0, 5): '█', (0, 3): '█', (0, 0): '█', (1, 1): '█', (1, 2): '█', (1, 3): '█', (1, 4): '█', (1, 0): '█', (1, 5): '█', (1, 6): '█', (2, 1): '░', (2, 4): '░', (2, 2): '░', (2, 6): '░', (2, 3): '█', (2, 0): '░', (2, 5): '░', (3, 4): '█', (3, 3): '█', (3, 2): '█', (4, 2): '█', (4, 3): '░', (4, 4): '█', (5, 1): '█', (5, 5): '█', (5, 4): '█', (5, 2): '█', (6, 2): '░', (6, 1): '█', (6, 4): '░', (6, 5): '█', (7, 0): '█', (7, 1): '█', (7, 5): '█', (7, 6): '█', (8, 6): '█', (8, 5): '░', (8, 1): '░', (8, 0): '█', (9, 0): '█', (9, 6): '█', (10, 6): '░', (10, 0): '░', (12, 6): '█', (12, 0): '█', (13, 6): '█', (13, 0): '█', (14, 6): '█', (14, 4): '█', (14, 2): '█', (14, 0): '█', (14, 1): '█', (14, 5): '█', (14,